In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import seaborn as sns
import numpy as np

%matplotlib inline

# Load Datasets

In [ ]:
# Base raw URL from GitHub (must be the "raw" version)
base_url = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/'

demo= 'df_final_demo.txt'
clients ='df_final_experiment_clients.txt'
web1= 'df_final_web_data_pt_1.txt'
web2= 'df_final_web_data_pt_2.txt'

df_demo = pd.read_csv(base_url + demo, delimiter=',')
df_client = pd.read_csv(base_url + clients, delimiter=',')
df_web1 = pd.read_csv(base_url + web1, delimiter=',')
df_web2 = pd.read_csv(base_url + web2, delimiter=',')

# Merge web data
df_web = pd.concat([df_web1, df_web2], ignore_index=True)

### Cleaning functions application

In [ ]:
from cleaning_functions_05 import main_cleaning
main_cleaning(df_demo)

In [ ]:
from cleaning_functions_05 import apply_to_column, standardize_gender

df_demo = apply_to_column(df_demo, 'gender', standardize_gender)


In [ ]:
from cleaning_functions_05 import conv_datetime, merge_datasets

# Then call directly:
df_web = conv_datetime(df_web)


### Quick_data_reports

In [ ]:
from cleaning_functions_05 import quick_data_report
quick_data_report(df_demo)

In [ ]:
from cleaning_functions_05 import quick_data_report
quick_data_report(df_client)

In [ ]:
from cleaning_functions_05 import quick_data_report
quick_data_report(df_web)

# Handle Missing Values

In [ ]:
df_demo.dropna(inplace=True)

In [ ]:
df_client['Variation']= df_client['Variation'].fillna('Unknown')

# Merge datasets

In [ ]:
# Merge all three
df_final = merge_datasets(df_demo, df_client, df_web)

# Quick check
print(df_final.shape)
df_final.head()

#### Answer the following questions about demographics:

Who are the primary clients using this online process? 

- Are the primary clients younger or older, new or long-standing?

- Next, carry out a client behaviour analysis to answer any additional relevant questions you think are important.

#### We define primary users as those who:

Have high login frequency (logons_6_mnth)

Progress through **multiple 'process_steps'

Possibly have repeated web activity (visit_id or visitor_id)

### 1.1. Compare login frequency by age group:


In [ ]:
# Discretizing 'age' into 4 categories
bins = [18, 30, 45, 60, 75, 100]
labels = ['18-30', '31-45', '46-60', '61-75', '76+']
df_final['age_group'] = pd.cut(df_final['clnt_age'], bins=bins, labels=labels, include_lowest=True)


In [ ]:
# Step 1: Aggregate the data into a DataFrame
df_avgage = df_final.groupby('age_group', observed=True)['logons_6_mnth'].mean().reset_index()

# Step 2: Plot using Seaborn
cubehelix_colors = sns.cubehelix_palette(start=.5, rot=-.75, n_colors=len(df_avgage))
sns.set_palette(cubehelix_colors)
# Theme color
theme_color = "#711164"

fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(data=df_avgage, x='age_group', y='logons_6_mnth',   palette=cubehelix_colors,
    legend=False,   ax=ax)

# Step 3: Styling for presentation
ax.set_title('Avg Logons by Age Group', color=theme_color)
ax.set_xlabel('Age Group', color=theme_color)
ax.set_ylabel('Average Logons', color=theme_color)
ax.tick_params(colors=theme_color)

for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_linewidth(1.2)
    spine.set_color(theme_color)

plt.tight_layout()
plt.savefig("../slides/avg_logons_by_age.png", transparent=True, format="png")
plt.show()


### 1.2. Compare login frequency by tenure:


In [ ]:
#1.2. Compare login frequency by tenure:
df_final['tenure_years'] = df_final['clnt_tenure_yr'] + df_final['clnt_tenure_mnth'] / 12
df_final['tenure_group'] = pd.cut(df_final['tenure_years'], bins=[0, 2, 5, 10, 20, 40], labels=['<2y', '2-5y', '5-10y', '10-20y', '20y+'])

df_tenure = df_final.groupby('tenure_group', observed= True)['logons_6_mnth'].mean().reset_index()


In [ ]:
# Generate cubehelix palette with ENOUGH colors
n_colors = df_tenure['tenure_group'].nunique()
cubehelix_colors = sns.cubehelix_palette(start=.2, rot=-.32, n_colors=n_colors)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(
    data=df_tenure,
    x='tenure_group',
    y='logons_6_mnth',
    palette=cubehelix_colors,
    legend=False,                 
    ax=ax
)

# Theme styling
theme_color = "#711164"
ax.set_title('Avg Logons by Tenure Years', color=theme_color)
ax.set_xlabel('Tenure Years', color=theme_color)
ax.set_ylabel('Average Logons', color=theme_color)
ax.tick_params(colors=theme_color)

for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_linewidth(1.2)
    spine.set_color(theme_color)

plt.tight_layout()
plt.savefig("../slides/avg_logons_by_tenure.png", transparent=True, format="png")
plt.show()


# 2.Client Behaviour Analysis
#### 2.1. Do high-balance clients engage more digitally?
#### 2.2 Do highly engaged digital clients call less?
#### 2.3. How many process steps do they complete?

In [ ]:
# 2.1 Balance group and engagement digitally
df_final['balance_group'] = pd.qcut(df_final['bal'], q=4, labels=['Low', 'Mid', 'Mid_High', 'High'])
df_balances = df_final.groupby('balance_group', observed=True)['logons_6_mnth'].mean().reset_index()

# Generate cubehelix palette
cubehelix_colors = sns.cubehelix_palette(start=.6, rot=-.32, n_colors=len(df_balances))

# Plot using Seaborn
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(
    data=df_balances,
    x='balance_group',
    y='logons_6_mnth',
    hue='balance_group',              
    palette=cubehelix_colors,
    legend=False,                    
    ax=ax
)

# Style elements
theme_color = "#711164"
ax.set_title('Avg Logons by Balance Level', color=theme_color)
ax.set_xlabel('Balance Groups', color=theme_color)
ax.set_ylabel('Average Logons', color=theme_color)
ax.tick_params(colors=theme_color)

for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_linewidth(1.2)
    spine.set_color(theme_color)

plt.tight_layout()
plt.savefig("../slides/avg_logons_by_balance.png", transparent=True, format="png")
plt.show()



In [ ]:
# 2.2 Do highly engaged digital clients call less?
sns.scatterplot(data=df_final, 
                x='logons_6_mnth',
                y='calls_6_mnth',
                color='seagreen')  # Or any other color

plt.title("Logons vs Calls (6 Months)")
plt.tight_layout()
plt.savefig("../slides/logons_by_calls.png", transparent=True, format="png")
plt.show()


In [ ]:
#2.3.How many process steps do they complete?
# Count how many unique process steps each client completed
df_steps = df_final.groupby('client_id')['process_step'].nunique().reset_index()
df_steps.columns = ['client_id', 'steps_completed']

# # Merge steps back into the full dataset
# df_final = df_final.merge(df_steps, on='client_id', how='left')
# df_final['steps_completed'].fillna(0)

# Plot: Average number of steps completed per age group
df_final.groupby('age_group', observed=True)['steps_completed'].mean().plot(
    kind='bar',
    title='Avg Steps Completed by Age Group',
    color=cubehelix_colors
)

plt.ylabel('Avg Steps Completed')
plt.xlabel('Age Group')
plt.tight_layout()
plt.savefig("../slides/avaragestepsbyage.png", transparent=True, format="png")
plt.show()


In [ ]:
#2.4. Completed process by balance group
# Plot: Average number of steps completed per balance group
df_final.groupby('balance_group', observed=True)['steps_completed'].mean().plot(
    kind='bar',
    title='Avg Steps Completed by balance Group',
    color=cubehelix_colors[2]
)

plt.ylabel('Avg Steps Completed')
plt.xlabel('Balance Group')
plt.tight_layout()
plt.savefig("../slides/av_steps_by_balance.png", transparent=True, format="png")
plt.show()


In [ ]:
#2.5. Completed process by tenure group
# Plot: Average number of steps completed per balance group
df_final.groupby('tenure_group', observed=True)[['logons_6_mnth','steps_completed']].mean().plot(
    kind='bar',
    title='Avg Steps Completed by Tenure Years')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.ylabel('Avg Steps Completed')
plt.xlabel('Tenure Years')
plt.tight_layout()
plt.savefig("../slides/av_steps_by_tenure.png", transparent=True, format="png")
plt.show()

# Boxplots

# Examine correlations between numerical features

In [ ]:
df_final.columns

In [ ]:
corr_matrix = df_demo.select_dtypes(include='number').corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

# Barplots